In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,508.600000,5.1990,3414.093682,63.40,NaN,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN,NaN
DROSND-TURBND,209.800000,0.4000,2859.669112,14.76,NaN,NaN
SMESHEND,197.800000,0.5490,2848.521603,78.16,NaN,NaN
DOOTB2,131.400000,0.2542,2724.861533,78.16,NaN,NaN
DOOTB1,NaN,0.1525,2652.000000,78.16,NaN,NaN
INCND,NaN,0.1525,2652.000000,78.16,NaN,NaN
INKOND,41.510000,0.0017,2338.024560,78.16,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей


Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0, maxiterations=50, calctolerance=10**-5)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

353.36844936030184
36.73590229404478
0
1.0
412.9325164466777
50.976300140664485
0
1.0
412.19025306012327
50.7863876434497
[0.8493756526634736, 0.8372321641331486, 0.8149684642884961, 43.889920248530075]
[0.8493807611371985, 0.8372415159994411, 0.8149716606831633, 0.726438272907929]
fin:--- 0.1 сек. ---


In [3]:
# надо посчитать потерю давления НД (готово)
# отборы неправильно (второй отбор завышен, надопроверить исходные данные функции)
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,505.067313,7.823193,3414.093682,63.40,6.755086,100.000000
ENDOFVD,193.369381,0.547199,2838.966896,63.40,6.984686,100.000000
DROSND-TURBND,204.357055,0.609199,2859.669112,14.76,6.980574,100.000000
SMESHEND,195.160981,0.547199,2842.876373,78.16,6.993051,100.000000
DOOTB2,128.810700,0.253700,2719.197411,78.16,7.052608,100.000000
DOOTB1,111.876550,0.152654,2646.670637,78.16,7.094448,97.876852
INCND,111.876550,0.152654,2646.670637,78.16,7.094448,97.876852
INKOND,26.850000,0.013031,-13365.212696,78.16,-3745.671786,100.000000


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,41.510000,0.0080,2341.254319,78.16,NaN


In [5]:
Get_value='P'
Error=(water_streams['DROSVD-TURBVD':'INKOND'][Get_value]-water_streams0['DROSVD-TURBVD':'INKOND'][Get_value])/water_streams0['DROSVD-TURBVD':'INKOND'][Get_value]*100
Error

DROSVD-TURBVD    -0.277975
ENDOFVD          -0.327977
DROSND-TURBND    -0.294696
SMESHEND         -0.327977
DOOTB2           -0.196675
DOOTB1            0.101041
INCND             0.101041
INKOND           62.885339
Name: P, dtype: float64

In [6]:
Turb.calculate_power()

{'Ni': 1303.2872799042996,
 'Ni1': 36.46303824243418,
 'Ni2': 9.666747644105397,
 'Ni3': 1251.4888013352154,
 'Ni4': 5.668692682544592}